In [ ]:
import sqlite3
import datetime as datetime
import os
import itertools
import numpy as np
import pandas as pd
conn = sqlite3.connect("testDB.db")

In [ ]:
abstract=pd.read_sql_query("select doi,abstract from trial_10000;", conn)

In [ ]:
abstract.head()

In [ ]:
abstract['abstract']=abstract['abstract'].str.replace('\d+', '') # for digits
abstract['abstract']=abstract['abstract'].str.replace(r'(\b\w{1,2}\b)', '') # for words
abstract['abstract']=abstract['abstract'].str.replace('Background', '')

In [ ]:
abstract.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words='english',)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem

english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([english_stemmer.stem(w) for w in analyzer(doc)])

vectorizer_s = StemmedCountVectorizer(min_df=3, analyzer="word", stop_words='english')

In [ ]:
vectorizer_s.fit(abstract['abstract'])

In [ ]:
vectorizer_s.get_feature_names()[0:5]

In [ ]:
dtm = vectorizer_s.fit_transform(abstract['abstract']).toarray()

In [ ]:
vocab = np.array(vectorizer_s.get_feature_names())

In [ ]:
dtm.shape

In [ ]:
len(vocab)

In [ ]:
from sklearn import decomposition

In [ ]:
num_topics = 200
num_top_words = 200
clf = decomposition.NMF(n_components=num_topics, random_state=1)

In [ ]:
doctopic = clf.fit_transform(dtm)

In [ ]:
topic_words = []

for topic in clf.components_:
    word_idx = np.argsort(topic)[::-1][0:num_top_words]
    topic_words.append([vocab[i] for i in word_idx])
  

In [ ]:
doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)

In [ ]:
doc_names=np.asarray(abstract['doi'])

In [ ]:
print(doc_names)

In [ ]:
doctopic_orig = doctopic.copy()

In [ ]:
num_groups = len(set(doc_names))

In [ ]:
doctopic_grouped = np.zeros((num_groups, num_topics))

In [ ]:
for i, name in enumerate(sorted(set(doc_names))):
    doctopic_grouped[i, :] = np.mean(doctopic[doc_names == name, :], axis=0) 

In [ ]:
plos_articles = sorted(set(doc_names))

In [ ]:
for i in range(len(doctopic_grouped)):
    top_topics = np.argsort(doctopic_grouped[i,:])[::-1][0:10]
    top_topics_str = ' '.join(str(t) for t in top_topics)
    print("{}: {}".format(plos_articles[i], top_topics_str))

In [ ]:
 for t in range(len(topic_words)):
        print("Topic {}: {}".format(t, ' '.join(topic_words[t][:15])))

In [ ]:
basket=[]
for i in range(len(doctopic_grouped)):
    top_topics = np.argsort(doctopic_grouped[i,:])[::-1][0:10]
    basket.append(list(top_topics))
print(basket)

In [ ]:
import pandas as pd
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import apriori

oht = OnehotTransactions()
oht_ary = oht.fit(basket).transform(basket)
df = pd.DataFrame(oht_ary, columns=oht.columns_)
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)

frequent_itemsets

In [ ]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)